## Extracción de datos de la API de idealista

In [2]:
import requests
from base64 import b64encode

def get_access_token(api_key, api_secret):
    '''creo una función para acceder al token según las instrucciones'''
    credentials = f"{api_key}:{api_secret}"
    encoded_credentials = b64encode(credentials.encode()).decode('utf-8')

    token_url = "https://api.idealista.com/oauth/token"

    token_headers = {
        'Authorization': f'Basic {encoded_credentials}',
        'Content-Type': 'application/x-www-form-urlencoded;charset=UTF-8'}

    token_data = {
        'grant_type': 'client_credentials',
        'scope': 'read'}

    token_response = requests.post(token_url, headers=token_headers, data=token_data)

    if token_response.status_code == 200:
        token_json = token_response.json()
        access_token = token_json['access_token']
        return access_token
    else:
        print(f"Error: {token_response.status_code}")
        print(token_response.text)
        return None
    


def search_properties(api_key, api_secret, country, operation, property_type, center, distance, locale='es', max_items=50, num_page=1, order='publicationDate', sort='desc', **filters):
    access_token = get_access_token(api_key, api_secret)
    '''Creo la función donde hago la llamada segun los filtros de busqueda que desee aplicar'''
    if not access_token:
        return None

   
    search_url = f"https://api.idealista.com/3.5/{country}/search"

    
    headers = {
        'Authorization': f'Bearer {access_token}',
        'Content-Type': 'application/x-www-form-urlencoded;charset=UTF-8'}

    
    data = {
        'locale': locale,
        'maxItems': max_items,
        'numPage': num_page,
        'operation': operation,
        'order': order,
        'propertyType': property_type,
        'sort': sort,
        'center': center,
        'distance': distance,}

    # para añadir filtros adicionales
    data.update(filters)

    response = requests.post(search_url, headers=headers, data=data)

    if response.status_code == 200:
        json_response = response.json()
        return json_response
    else:
        print(f"Error: {response.status_code}")
        print(response.text)
        return None



In [80]:
#creo las variables de busqueda
api_key = "bd8su7emxovedjjbw00sz1vu8kgluvwc"
api_secret = "yx07uav2PwqX"
country = "es"
operation = "rent"
property_type = "homes"
center = "37.178,-5.851"
distance = "9000"


In [ ]:
import pandas as pd

In [76]:
numero_pagina = [1]

In [81]:
resultados = {}
for numero in numero_pagina:
    nombre_variable = f"result_{numero}"
    resultados[nombre_variable] = search_properties(api_key, api_secret, country, operation, property_type, center, distance,num_page=numero)

In [82]:
dataframes = {}
for numero in numero_pagina:
    nombre_dataframe = f"df_{numero}"
    dataframes[nombre_dataframe] = pd.DataFrame(resultados[f'result_{numero}']['elementList'])

In [84]:
for key, df in dataframes.items():
    df_total = pd.concat([df_total, df], axis=0)

In [87]:
df_total.to_csv('data/raw/df_total.csv')

In [86]:
df_total

,propertyCode,thumbnail,numPhotos,floor,price,propertyType,operation,size,exterior,rooms,bathrooms,address,province,municipality,district,country,latitude,longitude,showAddress,url,distance,description,hasVideo,status,newDevelopment,hasLift,priceByArea,detailedType,suggestedTexts,hasPlan,has3DTour,has360,hasStaging,topNewDevelopment,superTopHighlight,externalReference,neighborhood,labels,parkingSpace,highlight,newDevelopmentFinished
0,101934757,https://img3.idealista.com/blur/WEB_LISTING/0/...,14,bj,590.0,flat,rent,66.0,True,2,1,Calle LOPE DE VEGA,Sevilla,Dos Hermanas,El Arenal - La Pólvora,es,37.284635,-5.925679,False,https://www.idealista.com/inmueble/101934757/,12321,"Hola soy Maite, ALQUILAMOS apartamento céntric...",True,good,False,True,9.0,{'typology': 'flat'},"{'subtitle': 'El Arenal - La Pólvora, Dos Herm...",False,False,False,False,False,False,NaN,NaN,NaN,NaN,NaN,NaN
1,103193364,https://img3.idealista.com/blur/WEB_LISTING/0/...,12,1,800.0,flat,rent,85.0,True,3,1,barrio Gran Plaza - Marqués de Pickman - Ciuda...,Sevilla,Sevilla,Nervión,es,37.388130,-5.963097,False,https://www.idealista.com/inmueble/103193364/,1686,Espléndido piso de 85 m² situado en Nervión. J...,False,good,False,True,9.0,{'typology': 'flat'},{'subtitle': 'Gran Plaza - Marqués de Pickman ...,False,False,False,False,False,False,NER 187,Gran Plaza - Marqués de Pickman - Ciudad Jardín,NaN,NaN,NaN,NaN
2,103192622,https://img3.idealista.com/blur/WEB_LISTING/0/...,1,bj,650.0,flat,rent,41.0,True,1,1,Calle Antonio Filpo Rojas s/n,Sevilla,Sevilla,Santa Justa - Miraflores - Cruz Roja,es,37.396532,-5.976552,True,https://www.idealista.com/inmueble/103192622/,1266,¡No pierdas la oportunidad de vivir en este ac...,False,good,False,False,16.0,{'typology': 'flat'},"{'subtitle': 'Arroyo - Santa Justa, Sevilla', ...",False,False,False,False,False,False,NaN,Arroyo - Santa Justa,NaN,NaN,NaN,NaN
3,99228868,https://img3.idealista.com/blur/WEB_LISTING/0/...,20,bj,650.0,flat,rent,40.0,True,1,1,"Calle Cardenal Rodrigo de Castro, 28",Sevilla,Sevilla,Nervión,es,37.377477,-5.961966,True,https://www.idealista.com/inmueble/99228868/,2007,Estupendo apartamento totalmente reformado y a...,False,good,False,False,16.0,{'typology': 'flat'},{'subtitle': 'Gran Plaza - Marqués de Pickman ...,False,False,False,False,False,False,CUIDAD JARDIN,Gran Plaza - Marqués de Pickman - Ciudad Jardín,"[{'name': 'apartamentoType', 'text': 'Apartame...",NaN,NaN,NaN
4,103188131,https://img3.idealista.com/blur/WEB_LISTING/0/...,18,NaN,1000.0,chalet,rent,256.0,NaN,3,3,Condequinto,Sevilla,Montequinto,Condequinto,es,37.349163,-5.935799,False,https://www.idealista.com/inmueble/103188131/,5783,"Casa de alquiler en Condequinto, de dos planta...",False,good,False,NaN,4.0,"{'typology': 'chalet', 'subTypology': 'indepen...","{'subtitle': 'Condequinto, Montequinto', 'titl...",False,False,False,False,False,False,LCSE27440,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20,102103782,https://img3.idealista.com/blur/WEB_LISTING/0/...,8,bj,500.0,flat,rent,84.0,True,3,2,barrio El Tinte,Sevilla,Utrera,Utrera,es,37.175795,-5.786509,False,https://www.idealista.com/inmueble/102103782/,5718,"1. Piso → Piso en Utrera zona El Tinte, 84 m. ...",False,good,False,True,6.0,{'typology': 'flat'},"{'subtitle': 'El Tinte, Utrera', 'title': 'Piso'}",False,False,False,False,False,False,0220323,El Tinte,NaN,"{'hasParkingSpace': True, 'isParkingSpaceInclu...",NaN,NaN
21,101870980,NaN,0,bj,600.0,flat,rent,83.0,True,3,1,barrio Salesianos,Sevilla,Utrera,Utrera,es,37.179141,-5.775776,False,https://www.idealista.com/inmueble/101870980/,6665,NaN,False,good,False,False,7.0,{'typology': 'flat'},"{'subtitle': 'Salesianos, Utrera', 'title': 'P...",False,False,False,False,False,False,0220315,Salesianos,NaN,NaN,NaN,NaN
22,101336250,NaN,0,NaN,600.0,chalet,rent,130.0,NaN,3,1,barrio Salesianos,Sevilla,Utrera,Utrera,es